In [3]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [4]:
%run PARAMETERS.py
NAME = 'APG'
PATH = NAME + '-bmnist-3digits-galileo-96'
CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:1')

from model_operations import Init_models_test, Save_models

models, AT = Init_models_test(K=K,
                                D=D,
                                FRAME_PIXELS=FRAME_PIXELS, 
                                DIGIT_PIXELS=DIGIT_PIXELS, 
                                HIDDEN_LIST=HIDDEN_LIST, 
                                Sigma0=Sigma0,
                                CUDA=CUDA, 
                                DEVICE=DEVICE, 
                                lr=LEARNING_RATE, 
                                RESTORE=True, 
                                PATH=PATH)

mnist_mean = mnist_mean.cuda().to(DEVICE)

In [5]:
K=4
NUM_GROUPS_TEST = 1
T_test =20
data_path_test = "/data/hao/apg_data/mnist/test_T%dD%d/" % (T_test, K)


In [6]:
from apg_galileo import APG
from viz_mc import Viz_MC
from testing import Eval
BATCH_SIZE = 5
Viz_INTERVAL = 5
FS = 30*1.5
TITLE_FS = 20
LW= 3

COLORS = ['#AA3377', '#EE7733', '#009988', '#0077BB', '#BBBBBB', '#EE3377']
apg = APG(models=models, 
          frame_size=FRAME_PIXELS,
          AT=AT, 
          K=K, 
          D=D, 
          T=T_test, 
          mnist_mean=mnist_mean, 
          training=False)

evaluator = Eval(DATA_PATH=data_path_test,
                 NUM_GROUPS=NUM_GROUPS_TEST, 
                 K=K, 
                 D=D)

# visualizer = Viz_MC(K=K, 
#                     viz_interval=Viz_INTERVAL, 
#                     FS=FS, 
#                     TITLE_FS=TITLE_FS, 
#                     COLORS=COLORS,
#                     LW=LW)


In [11]:
def Test_uniform(APG, group_ptr, data_ptr, APG_STEPS, S, B, DEVICE, PATH):
    """
    training function
    """

    group_ind = torch.arange(NUM_GROUPS_TEST)
    data_g = torch.from_numpy(np.load(data_path_test + 'ob_%d.npy' % group_ind[group_ptr])).float()
    NUM_DATA= data_g.shape[0]
    indices = torch.arange(NUM_DATA)
    MSEs = []
    LJs = []
    NUM_BATCHES = int(NUM_DATA / B)
    for b in range(NUM_BATCHES):
#         optimizer.zero_grad()
        time_start = time.time()
        frames = data_g[data_ptr*B : (data_ptr+1)*B]
#         Metrics['data'] = frames
        frames = frames.repeat(S, 1, 1, 1, 1).cuda().to(DEVICE)
        metrics = APG.Sweeps(APG_STEPS, S, B, frames)
        MSE = ((metrics['recon'].cpu() - frames.cpu())**2).sum(-1).sum(-1).mean(-1).mean(-1).mean(-1)
        MSEs.append(MSE.unsqueeze(0))
        time_end = time.time()
        print('b=%d, (%ds)' % (b, time_end - time_start))
        if b == 99:
            break
    MSEs = torch.cat(MSEs, 0)
    return MSEs

In [12]:
# DATA_PTR = 1
GROUP_PTR = 2
DATA_PTR= 4

APG_STEPS = 10

MSEs = Test_uniform(APG=apg, 
                        group_ptr=0,
                        data_ptr=DATA_PTR,
                        APG_STEPS=APG_STEPS, 
                        S=50,
                        B=BATCH_SIZE,
                        DEVICE=DEVICE,
                        PATH=PATH)

np.save('mse-T=%d-D=%d' % (T_test, K), MSEs.data.numpy())

b=0, (12s)
b=1, (13s)
b=2, (13s)
b=3, (17s)
b=4, (17s)
b=5, (17s)
b=6, (17s)
b=7, (18s)
b=8, (19s)
b=9, (17s)
b=10, (17s)
b=11, (18s)
b=12, (17s)
b=13, (18s)
b=14, (18s)
b=15, (18s)
b=16, (17s)
b=17, (17s)
b=18, (17s)
b=19, (17s)
b=20, (19s)
b=21, (18s)
b=22, (17s)
b=23, (17s)
b=24, (18s)
b=25, (17s)
b=26, (18s)
b=27, (18s)
b=28, (18s)
b=29, (17s)
b=30, (17s)
b=31, (17s)
b=32, (17s)
b=33, (18s)
b=34, (18s)
b=35, (17s)
b=36, (17s)
b=37, (17s)
b=38, (17s)
b=39, (18s)
b=40, (18s)
b=41, (18s)
b=42, (17s)
b=43, (17s)
b=44, (17s)
b=45, (17s)
b=46, (18s)
b=47, (19s)
b=48, (18s)
b=49, (17s)
b=50, (17s)
b=51, (17s)
b=52, (18s)
b=53, (18s)
b=54, (18s)
b=55, (17s)
b=56, (17s)
b=57, (17s)
b=58, (17s)
b=59, (18s)
b=60, (18s)
b=61, (17s)
b=62, (17s)
b=63, (18s)
b=64, (17s)
b=65, (18s)
b=66, (18s)
b=67, (18s)
b=68, (17s)
b=69, (17s)
b=70, (17s)
b=71, (18s)
b=72, (18s)
b=73, (18s)
b=74, (17s)
b=75, (18s)
b=76, (17s)
b=77, (17s)
b=78, (18s)
b=79, (19s)
b=80, (18s)
b=81, (17s)
b=82, (17s)
b=83, (17s)
b=

In [13]:
np.save('mse-T=%d-D=%d' % (T_test, K), MSEs.data.numpy())